In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [2]:
from joblib import Parallel, delayed
import ARX_Optimization 
import pandas
import numpy as np
import xlsxwriter
import pathlib
import configparser
from config import input_datadir, result_datadir
import matplotlib.pyplot as plt
import os
import fnmatch

In [3]:
for filename in os.listdir(input_datadir):
        if filename.endswith('.csv'):
            df = pandas.read_csv(input_datadir/filename)
            ydata = df.T1
            udata = df.Q1
            times = df.Time
            
            # Adding 10 degrees of noise:
            otpts, std_dev = ARX_Optimization.add_noise(ydata, 5)

In [52]:
std_dev[9], len(otpts), otpts[10,:]

(5.0, 11, array([22.03798087, 18.67142355, 22.81314973, ..., 51.34667596,
        48.64740394, 51.86777716]))

In [57]:
N = 'TEST'
m = n = 2

for method_name, method in ARX_Optimization.__dict__.items():
    if callable(method) and fnmatch.fnmatch(method_name, '*coeffs'):
        if fnmatch.fnmatch(method_name, 'DE*'):
            runtimes = []
            coefficients = []
            for i in range(len(otpts)):
                ni = otpts[i,:]
                bnds = tuple((1,1) for j in range(m+n))
                glob_args = bnds, ni, udata, m, n
                rts = 'rt'
                coeffs = np.array(['a1', 'a2', 'b1', 'b2'])
#                 rts, coeffs = method(*glob_args)
                runtimes.append(rts)
                coefficients.append(coeffs)
        
        else:
            runtimes = []
            coefficients = []
            for i in range(len(otpts)):
                ni = otpts[i,:]
                initial_guess = [1 for k in range(m+n)]
                int_args = initial_guess, ni, udata, m, n
                rts = 'rt'
                coeffs = np.array(['a1', 'a2', 'b1', 'b2'])
#                 rts, coeffs = method(*int_args)
                runtimes.append(rts)
                coefficients.append(coeffs)

        new_filename = os.path.splitext(filename)[0]+str(method_name)+'_m'+str(m)+'_n'+str(n)+"_"+str(N)+'.xlsx'
        workbook = xlsxwriter.Workbook(result_datadir/new_filename)
        worksheet1 = workbook.add_worksheet("Input_Data")
        worksheet2 = workbook.add_worksheet("ID_Results")

        col = 0
        for set_ in otpts:
            row = 0
            for value, t in zip(set_, times):
                worksheet1.write(row, 0, t)
                worksheet1.write(row, col, value)
                row +=1
            col+=1
        row1=0
        for cf, rt, sig in zip(coefficients, runtimes, std_dev):
            col1=0
            for c in cf:
                worksheet2.write(row1, col1, c)
                col1+=1
            worksheet2.write(row1, col1, rt)
            worksheet2.write(row1, col1+1, sig)
            row1+=1
        workbook.close()



In [17]:
bnds

((1, 1), (1, 1), (1, 1), (1, 1))

In [12]:
import scipy.optimize

In [23]:
import numpy as np
def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e
bounds = [(-5, 5), (-5, 5)]
result = scipy.optimize.differential_evolution(ackley, bounds).x
result

array([0., 0.])